In [1]:
# ЧЕРНОВОЙ ВАРИАНТ ПО НЕЙРО-КОПИРАЙТЕРУ PDF С НЕСУЩЕСТВУЮЩИМИ ТАБЛИЦАМИ С ПОМОЩЬЮ  В COLAB

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -rf "/content/drive/MyDrive/KIA_PDF/Copywriter/KIA/BASE/"
!mkdir -p "/content/drive/MyDrive/KIA_PDF/Copywriter/KIA/BASE/"

In [3]:
KEY="BASE_2"
ALL_ISSUE_TEXTS={"BASE_1" : "https://drive.google.com/file/d/1bB4_eV6Hn3jj-xBBBg1-NR6YcriVMXyZ/view?usp=drive_link"
,"BASE_2" : "https://drive.google.com/file/d/1ICfdsJN19JOoSDV09Y67dSiyFuwDTZRV/view?usp=drive_link"
,"BASE_3" : "https://drive.google.com/file/d/1wkajJd1smmR8WTVL4LN1vrTY_MbteXG7/view?usp=drive_link"
,"BASE_4" : "https://drive.google.com/file/d/1u0B54aQcm4ujvhxBfJ_2cm5RUD9iwPRR/view?usp=drive_link"
}
ISSUE_TEXTS={KEY: ALL_ISSUE_TEXTS[KEY]}

In [4]:
#@title Установка библиотек
!pip install tiktoken langchain openai chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu python-docx

In [5]:
#@title Импорт библиотек
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

In [6]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Обработка pdf файлов Киа

# Задача создания нейро-копирайтера
## План создания нейро-копирайтера
###• Исходные материалы: видео
###• Распознавание речи через Whisper
###• Разделение текста на чанки по 3000 токенов [у нас 2000]
###• Саммаризация чанков до 1000 токенов [у нас 2000]
###• Составление заголовков и подзаголовков по 5-10-15 чанкам [у нас 15]
###• Улучшение списка заголовков
###• Составление текста для каждого заголовка/подзаголовка
###• Запуск нейро-куратора
###• Тестирование нейро-куратора

# Роль1 - Копирайтер - Системная роль
###• system_prompt = """
###• Ты профессиональный копирайтер. У тебя большой опыт работы с Бизнесом в разных
###• сферах, ты качественно структурируешь текст на Русском языке.
###• Сделай формальный технический пересказ того, о чем рассказывает специалист в
###• диалоге с другими клиентами. Пиши от имени Специалиста.
###• Необходимо уложиться в 2000 токенов.
###• """

#Роль1 - Копирайтер - Инструкции в messages
###• messages = [
###• {"role": "system", "content": f"{self.system_promt}"},
###• {"role": "user", "content": f"""
###• Проанализируй отрывок материала с семинара: {materials}.
###• дай компактный, сжатый, обобщенный список тем и подтем.
###• Темы Необходимо оформить #, а подтемы оформить ##_.
###• Используй только такой пример, ничего не добавляй лишнего.
###• Пример составления списка:
###• _#...
###• ##_...
###• _#...
###• ##...
###• """}
###• ]

# Роль2 - Специалист - системная роль
###• speaker_system_promt = """
###• Ты Специалист, готовящий текст для обучающего материала. Ты профессионал своего дела.
###• Тебе предоставят отрывок Твоего Текста и попросят раскрыть подробнее одну из тем
###• обучающего материала.
###• Твоя цель: Опираясь только на Текст, подробно, развернуто рассказать по
###• интересующей теме на русском языке.
###• Необходимо уложиться в 6000 токенов.
###• """


# Роль2 - Специалист - Инструкции в messages
###• messages = [
###• {"role": "system", "content": f"{self.speaker_system_promt}"),
###• {"role": "user", "content": f"""
###• Вот отрывки Вашего текста:\n{message_content}
###• Опираясь только на информацию из Вашего Текста, указанного выше, расскажите
###• подробнее по теме: {topic}.
###• """}
###• ]

In [7]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

from IPython.display import clear_output
clear_output()

In [8]:
import re
import os
import json
from tqdm.auto import tqdm
import getpass
from tqdm.auto import tqdm

# ----------------------------------
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"
# ----------------------------------

In [9]:
 # Переводим текстовый файл в word
import requests
import docx

def convert_txt_to_txt(url, output_path):
    match_ = re.search('/file/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    response = requests.get("https://docs.google.com/uc?export=download&id="+doc_id)
    encoding = response.encoding
    content = response.text

    with open(output_path, "w") as fw:
        fw.write(content.encode(encoding, 'ignore').decode('utf-8'))
        fw.write("\n")
    print(encoding)

BASE = "/content/drive/MyDrive/KIA_PDF/Copywriter/KIA/BASE/"
for key in ISSUE_TEXTS:
    convert_txt_to_txt(ISSUE_TEXTS[key], BASE + str(key) + ".txt")


ISO-8859-1


In [10]:
import time

class WorkerОpenAI():
  def __init__(self, \
               system_promt = " ", \
               system_promt_lector = " ", \
               mod = MODEL_TURBO_16K, \
               content_topics = None, \
               save_project = '/content/'):
    self.model = mod
    self.save_project  = save_project

    if content_topics:
      self.content_topics = self.load_txt_file(content_topics)

    # системные настройки
    self.system_promt = self.load_document_text(system_promt)
    self.speaker_system_promt = self.load_document_text(system_promt_lector)


  def load_document_text(self, url: str) -> str:
      # функция для загрузки документа по ссылке из гугл док
      match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
      if match_ is None:
          raise ValueError('Invalid Google Docs URL')
      doc_id = match_.group(1)
      response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
      response.raise_for_status()
      text = response.text
      return text


  def load_txt_file(self, file_path):
      with open(file_path, 'r') as file_:
          text = file_.read()
      return text

  # пример подсчета токенов
  def num_tokens_from_messages(self, messages):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(self.model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      # if self.model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      if self.model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-4-0613"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {self.model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


  def f00_create_embedding_faiss_db(self, doc_txt_dir="/content/", \
                                faiss_db_dir ="/content/", \
                                start_idx = 0, \
                                collection_name = " "):

    def f01_num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=1024, chunk_overlap=300)
    idx_file_folder = start_idx-1
    chunkID = idx_file_folder

    count_tokens = 0
    # проходимся по всем данным
    for _, file_ in enumerate(sorted(os.listdir(doc_txt_dir))):
        print("Загружается файл: ", file_)
        self.file_name = file_
        idx_file_folder +=1
        source_chunks = []
        # разбиваем на несколько частей с помощью метода split_text
        with open(doc_txt_dir + file_, "r") as f:
          for chunk in self.splitter.split_text(f.read()):
              # time.sleep(1)
              chunkID += 1
              source_chunks.append(Document(page_content=chunk, \
                                  metadata={'source': file_,
                                            'chunkID': chunkID,
                                            "collection_name": collection_name,
                                            'idx_file_folder': idx_file_folder}))


        # Создание индексов документа и СОХРАНЕНИЕ
        # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
        if len(source_chunks) > 0:
            self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
            count_token = f01_num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
            count_tokens += count_token
            print('Количество токенов в документе :', count_token)
            # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

            self.db.save_local(os.path.join(faiss_db_dir, collection_name, f"{str(idx_file_folder)}_db_initial__{file_[:20]}"))

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')

  # ЗАПРС в ChatGPT
  def get_ChatCompletion(self, model,  # указываем модель
                         messages,     # словарь запроса
                         temp=0.1):    # температуру

      completion = openai.ChatCompletion.create(
        model= model,
        messages= messages,
        temperature= temp
        )

      print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
      # print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
      print('===========================================: \n')
      return completion.choices[0].message.content

  # ЗАПРС на создание темы и подтемы по Материалам
  def get_search_materials_topics_subtopics(self, materials, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй отрывок материала из диалога с другими клиентами: {materials}.
• дай компактный, сжатый, обобщенный список тем и подтем.
• Темы Необходимо оформить #, а подтемы оформить ##_.
• Используй только такой пример, ничего не добавляй лишнего.
• Пример составления списка:
Пример составления списка:
_#...
##_...
_#...
##_...
        """}
        ]

    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")


  # ПОИСК ТЕМЫ и ПОДТЕМЫ
  def search_topics_subtopics(self, num_chunk = 10):
    self.content_topics = ''
    materials = ""
    # Выбираем блоки ПОДРЯД
    len_chunk = len(self.db.docstore._dict)
    if len_chunk < num_chunk + 1:
        for _, doc in self.db.docstore._dict.items():
            materials += f"{doc.page_content}\n"
        self.get_search_materials_topics_subtopics(materials)
    else:
        for ind, (key, doc) in enumerate(self.db.docstore._dict.items()):
          materials += f"{doc.page_content}\n"
          if (ind+1) % num_chunk == 0:
            self.get_search_materials_topics_subtopics(materials)
            materials = ""
    if materials != "":
        self.get_search_materials_topics_subtopics(materials)

    print('Собрали список тем и подтем: ')
    print(self.content_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы.txt', "w") as f:
      f.write(self.content_topics)


  # ОБЪЕДИНЯЕМ схожие ТЕМЫ
  def get_merge_topics(self, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй Темы и подтемы из Текста диалогов специалиста с другими клиентами: {self.content_topics}.

Необходимо объединить похожие по смыслу темы или подтемы, записать компактно.
При необходимости перефразировать тему или подтему. Дай корректный список.
Темы оформи _#, а подтемы оформи ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
"""}
]
    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics = self.get_ChatCompletion(model_topics, messages)
      print('Итоговый список тем и подтем: ')
      print(self.content_topics)
      with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы_ИТОГ.txt', "w") as f:
        f.write(self.content_topics)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

  # ОРГАНИЗУЕМ текст блоками
  def organize_text(self, topic, num_chunks, model_topics):

    # Выборка документов по схожести с подтемой
    docs = self.db.similarity_search(topic, k = num_chunks)
    message_content = '\n'.join([doc.page_content + '\n' for i, doc in enumerate(docs)])

    messages = [
        {"role": "system", "content": f"{self.speaker_system_promt}"},
        {"role": "user", "content": f"""Вот отрывки Вашей лекции:\n{message_content}

Опираясь только на информацию с Лекции, указанной выше, расскажите подробнее по Теме: {topic}.
"""}
      ]

    # example token count from the function defined above
    print('\n ===========================================: ')
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.final_text += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

  # ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ
  def organize_final_text(self, num_chunks = 10,
                          model_topics = "gpt-3.5-turbo-16k",
                          name = None,
                          db_path = None):
    if name:
      self.file_name = name

    if db_path:
      for curr_base in os.listdir(db_path):
          self.db = FAISS.load_local(os.path.join(db_path, curr_base), OpenAIEmbeddings())

    self.final_text = ""
    list_topics = self.content_topics.split('\n')
    # проходимся по списку тем и подтем
    for ind, topic in tqdm(enumerate(list_topics)):
        # тему просто записываем в итоговый текст
        if ("_#" in topic) and not("_#" in list_topics[ind+1]):
          self.final_text += f'\n{topic}\n'
        # подтему передаем в ChatGPT
        else:
          self.final_text += f'\n{topic}\n'
          self.organize_text(topic, num_chunks, model_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__final_text.txt', "w") as f:
      f.write(self.final_text)
    print("\nСоздали финальный документ для Базы Знаний (final_text)")


# Copywriter

In [11]:
#@title Создаем объект для работы Copywriter
projects_dir = '/content/drive/MyDrive/KIA_PDF/Copywriter/KIA/'

Promt_copywriter = "https://docs.google.com/document/d/1D9lpYTUfLLr-25pnvbHUXK9mXv-rPRiFW3s92UMnPoE/edit?usp=sharing"
Promt_lector = "https://docs.google.com/document/d/1Lf70xX7_brwNE7FItvQbNtf51yR47Fuam7SorkvUe8s/edit?usp=sharing"

# если темы созданы
# topics_final = "/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/___.txt"
# # Создаем объект для дообучения chatGPT
curator = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                       system_promt_lector = Promt_lector, # промт Лектора
                      #  content_topics = topics_final,
                       save_project = projects_dir)     # путь для сохранения готовых файлов

# Формируем базу по файлу txt
# путь к материалам
doc_txt_dir = projects_dir + 'BASE/'
db_initial = projects_dir

curator.f00_create_embedding_faiss_db(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                faiss_db_dir = db_initial,     # путь для сохранения исходной базы
                                start_idx = 0,                 # номер документа в базе
                                collection_name = 'KIA_db_initial')  # наименование коллекции

Загружается файл:  BASE_2.txt
Количество токенов в документе : 64857

ЦЕНА запроса создания базы индексов: 0.025942800000000002  $


In [12]:
# Смотрим созданные чанки
print(f"Текст разбит на чанки. Всего: {len(curator.db.docstore._dict.values())} шт.\n")
curator.db.docstore._dict.values()

Текст разбит на чанки. Всего: 160 шт.



dict_values([Document(page_content='РАСХОД ТОПЛИВА\nУсловный расход топлива, городской цикл (л/100 км) 12,9 12,7 12,7 15,7\nУсловный расход топлива, загородный цикл (л/100 км) 6,5 7,2 7,2 8,4\nУсловный расход топлива, смешанный цикл (л/100 км) 8,8 9,2 9,2 11\nВЫБРОСЫ CO2\nВыбросы СО2, городской цикл (г/км) 296 289 289 355\nВыбросы СО2, загородный цикл (г/км) 148 164 164 193\nВыбросы СО2, смешанный цикл (г/км) 202 210 210 252\nВЕСОВЫЕ ХАРАКТЕРИСТИКИ И ВМЕСТИМОСТЬ\nСнаряженная масса (кг) 1 898 1 971\nОбъем багажного отделения в обычном состоянии / со сложенными сиденьями (л) 406 / 1 114\nОбъем топливного бака (л) 60\nРАЗМЕРЫ\nДлина 4 830 мм Ширина 1 870 мм Высота 1 400 мм Колесная база 2 905 мм\nГарантия и сервис Kia Finance', metadata={'source': 'BASE_2.txt', 'chunkID': 0, 'collection_name': 'KIA_db_initial', 'idx_file_folder': 0}), Document(page_content='ВЕСОВЫЕ ХАРАКТЕРИСТИКИ И ВМЕСТИМОСТЬ\nСнаряженная масса (кг) 1 898 1 971\nОбъем багажного отделения в обычном состоянии / со сложенны

In [ ]:
#@title Поиск темы и подтемы выборкой по 5 - 15 чанков

# Подаем в цикле и просим составить список тем и подтем.
curator.search_topics_subtopics(num_chunk = 15)

6163 токенов использовано на вопрос 



In [ ]:
#@title Корректируем список тем и подтем (обобщаем, убираем дубли).
# Подаем сформированный Список тем и просим объединить похожие по смыслу темы или подтемы, записать компактно.
# При необходимости перефразировать тему или подтему.
curator.get_merge_topics()

## ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ

Выборка блоков документов по схожести с подтемой. Передаем собранные отрывки с Лекции и просим рассказать подробнее, опираясь на Лекцию.

In [ ]:
# запускаем сбору Итогового Текста
# name_file = 'Разбор нейро-сотрудников на chatGPT _ 21.08.2023'
# db_initial = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/УИИ_db_initial'

# curator.organize_final_text(name = name_file, db_path = db_initial)
curator.organize_final_text()

In [ ]:
# сформирован итоговый текст
curator.final_text

# MarkdownHeaderTextSplitter

In [ ]:
#@title функции
chat_manager_system = """
Ты Специалист, готовящий текст для обучающего материала. Ты профессионал своего дела.
Тебе предоставят отрывок Твоего Текста и попросят раскрыть подробнее одну из тем обучающего материала.
Твоя цель: Опираясь только на Текст, подробно, развернуто рассказать по интересующей теме на русском языке.
Необходимо уложиться в 6000 токенов.
"""

def get_chatgpt_answer(topic,  db, model = MODEL_TURBO_0613):
  # Выборка документов по схожести с вопросом
  docs = db.similarity_search(topic, k=4)
  message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n  ' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
  # print('message_content :\n ======================================== \n', self.message_content)

  messages = [
    {"role": "system", "content": f"{chat_manager_system}"},
    {"role": "user", "content": f"""Analyze step by step and give a detailed correct answer to the Student's question.\n
    Question:\n{topic}\n\nMaterials from the Webinar:\n{message_content}\n\nAnswer:"""}
    ]

  try:
    completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0.1
    )

    print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
    print('===========================================: \n')
    print('Ответ ChatGPT: ')
    print(completion.choices[0].message.content)
    # return completion.choices[0].message.content
  except:
    print("Модель в настоящее время перегружена. Попробуйте позже.")

def load_txt_file(file_path):
    with open(file_path, 'r') as file_:
        text = file_.read()
    return text

projects_dir = '/content/drive/MyDrive/KIA_PDF/Copywriter/KIA/'
name_file = KEY + '.txt__final_text.txt' # FIXME
final_text_dir = projects_dir + name_file

# Загружаем итоговый текст
final_text = load_txt_file(final_text_dir)


In [ ]:
#@title  Готовим документ MarkdownHeader по сформированному тексту
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("_#", "Header 1"),
    ("##_", "Header 2"),
    ("###_", "Header 3"),
    ("####_", "Header 4"),
    ("#####_", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(final_text)
md_header_splits

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=300
)

# Split
split = text_splitter.split_documents(md_header_splits)
db = FAISS.from_documents(split, OpenAIEmbeddings())

In [ ]:
split[1]

In [ ]:
topic1 = """
Почем сегодня KIA CEED?
"""

get_chatgpt_answer(topic1,  db)

In [ ]:
topic2 = """
Почем сегодня KIA KARNIVAL?
"""

get_chatgpt_answer(topic2,  db)

In [ ]:
topic3 = """
Почем сегодня AUDI A6?
"""

get_chatgpt_answer(topic3,  db)